# OR-Tools를 이용한 차량-승객 매칭 실습

이 노트북은 다음을 다룹니다.
- OR-Tools로 승객과 차량을 최적 매칭하는 방법(Assignment)
- 매칭 결과를 기반으로 deck.gl 시각화를 위한 `trips.json` 및 `passengers.json` 생성
- React 기반 뷰어(`simulation_vis`)에서 애니메이션으로 경로와 승객 Point 레이어 확인

실습 흐름
1) 데이터 로딩(승객/차량) → 2) 비용행렬 구성 → 3) OR-Tools 매칭 → 4) 경로/시간 생성 → 5) JSON 저장 → 6) 뷰어 실행


In [ ]:
# 1) 의존성 (최초 1회만)
%pip -q install ortools pandas numpy tqdm


In [42]:
# 2) 임포트/함수 정의 (수식/제약조건 쉽게 설명 추가)
import os, json, math, random
from typing import List, Tuple
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

random.seed(42)
np.random.seed(42)

# 거리(km) 계산 함수 (위도/경도를 받아 두 점 사이 거리를 km로 반환)
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = phi2 - phi1
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# 경로를 따라 이동하며 도착시각(분 단위) 리스트 생성
def build_simple_route_with_timestamp(points_lonlat: List[Tuple[float, float]], speed_km_per_h: float = 30.0, start_time_min: float = 0.0):
    speed_km_per_min = speed_km_per_h / 60.0
    route = points_lonlat
    times = [start_time_min]
    for i in range(1, len(points_lonlat)):
        lon1, lat1 = points_lonlat[i-1]
        lon2, lat2 = points_lonlat[i]
        dist_km = haversine_km(lat1, lon1, lat2, lon2)
        dur_min = dist_km / speed_km_per_min
        times.append(times[-1] + dur_min)
    return route, times

# 사용할 최적화 솔버(Runtime에 따라 CBC/SCIP 중 자동 선택)
def _create_mip_solver():
    for name in ["CBC_MIXED_INTEGER_PROGRAMMING", "SCIP"]:
        s = pywraplp.Solver.CreateSolver(name)
        if s is not None:
            return s, name
    return None, None

# ==== (핵심) 차량-승객 매칭: 목적함수/제약조건 초보자 설명 추가 ====
def solve_assignment_ortools(cost_matrix: np.ndarray, big_m: float = 1e6, use_greedy_backup: bool = True):
    """
    매칭 문제:
      - 목적: '총 비용(예: 거리/시간 등)'이 가장 작도록 승객-차량을 짝지음
      - 입력: cost_matrix[p, v] (p:승객, v:차량) = 승객 p와 차량 v가 만날 때 드는 비용
    [수식 설명]
      1. 변수 정의: 
         x[p, v] = 1 → p번 승객을 v번 차량에 할당, 0 → 할당하지 않음 (binary 변수)
      2. 목적함수(Objective):
         모든 할당된 쌍의 '비용' 합계가 최소화(MINIMIZE)
         → sum(cost_matrix[p, v] * x[p, v])
      3. 제약조건(Constraints):
         (1) 승객은 '딱 1대'의 차량에 할당(혹은 매칭 안될 수도 있음. 여기선 반드시 1대)
             ∑(v) x[p, v] == 1      (각 승객 p마다)
         (2) 차량도 '딱 1명의 승객'만 태움(혹은 매칭 안될 수도 있음. 여기선 반드시 1명)
             ∑(p) x[p, v] == 1      (각 차량 v마다)
      참고: 승객과 차량 수 다를 땐 cost_matrix를 직사각형→정사각형으로 padding(빈자리엔 큰 비용)
    """

    P, V = cost_matrix.shape
    if P == 0 or V == 0:
        return []

    # 승객/차량 수 맞추기 위해 정사각행렬로 확장(padding)
    N = int(max(P, V))
    padded = np.full((N, N), big_m, dtype=float)
    padded[:P, :V] = cost_matrix

    solver, backend = _create_mip_solver()
    matches = []
    if solver is not None:
        # [변수 선언] x[i, j] = 1이면 매칭 0이면 아닌 이진변수
        x = {(i, j): solver.IntVar(0, 1, f"x_{i}_{j}") for i in range(N) for j in range(N)}
        
        # [제약조건1] 각 승객(i)는 차량을 정확히 1대만 배정받음
        for i in range(N):
            solver.Add(solver.Sum([x[i, j] for j in range(N)]) == 1)
        # [제약조건2] 각 차량(j)은 정확히 1명만 태움
        for j in range(N):
            solver.Add(solver.Sum([x[i, j] for i in range(N)]) == 1)
        
        # [목적함수] 모든 x[i, j]에 대해 "비용합"이 최소가 되도록
        solver.Minimize(solver.Sum(padded[i, j] * x[i, j] for i in range(N) for j in range(N)))
        
        # [최적화 수행]
        status = solver.Solve()
        if status in (pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE):
            for i in range(N):
                for j in range(N):
                    # i<P, j<V (=패딩 아닌 원래 승객/차량 경우만 정답 사용)
                    if x[i, j].solution_value() > 0.5 and i < P and j < V:
                        matches.append((i, j))
        print({'backend': backend, 'status': int(status), 'P': P, 'V': V, 'N': N, 'matched': len(matches)})

    # [예외 상황 대비] 만약 최적화모델이 실패하면, 비용 낮은 순서로 그리디 매칭(backup)
    if use_greedy_backup and len(matches) == 0:
        used_p, used_v = set(), set()
        pairs = [(cost_matrix[i, j], i, j) for i in range(P) for j in range(V) if np.isfinite(cost_matrix[i, j])]
        pairs.sort()
        for _, i, j in pairs:
            if i not in used_p and j not in used_v:
                used_p.add(i); used_v.add(j)
                matches.append((i, j))
        print({'backend': 'greedy_backup', 'P': P, 'V': V, 'matched': len(matches)})

    return matches


In [37]:
# 3) 데이터 로딩 (없으면 합성)
PASSENGER_CSV = "passenger_data.csv"
N_PASSENGERS = 20

if os.path.isfile(PASSENGER_CSV):
    passengers_raw = pd.read_csv(PASSENGER_CSV)
else:
    center_lat, center_lon = 37.4517, 127.1306
    rows = []
    for pid in range(1, 101):
        ride_lat = center_lat + np.random.normal(0, 0.002)
        ride_lon = center_lon + np.random.normal(0, 0.002)
        alight_lat = ride_lat + np.random.normal(0, 0.003)
        alight_lon = ride_lon + np.random.normal(0, 0.003)
        ride_time = max(0, int(np.random.uniform(0, 60)))
        rows.append({'ID': pid,'ride_time': ride_time,'ride_lat': ride_lat,'ride_lon': ride_lon,'alight_lat': alight_lat,'alight_lon': alight_lon})
    passengers_raw = pd.DataFrame(rows)

for col in ['ID','ride_time','ride_lat','ride_lon','alight_lat','alight_lon']:
    if col not in passengers_raw.columns:
        raise ValueError(f"필수 컬럼 누락: {col}")

passengers = passengers_raw[['ID','ride_time','ride_lat','ride_lon','alight_lat','alight_lon']].copy()
passengers = passengers.sort_values('ride_time').head(N_PASSENGERS).reset_index(drop=True)

# 차량 더 흩뿌리기: 중심에서 일정 반경(약 0.005~0.02도) 원형 분포로 초기화
M_VEHICLES = 30
center_lat = passengers['ride_lat'].mean(); center_lon = passengers['ride_lon'].mean()
vehicles = []
for vid in range(M_VEHICLES):
    angle = np.random.uniform(0, 2*np.pi)
    radius = np.random.uniform(0.005, 0.02)
    dlat = radius * np.sin(angle)
    dlon = radius * np.cos(angle)
    lat = center_lat + dlat
    lon = center_lon + dlon
    vehicles.append({'vehicle_id': vid, 'lat': lat, 'lon': lon, 'start_time': 0})
vehicles = pd.DataFrame(vehicles)

print({'P': len(passengers), 'V': len(vehicles)})
passengers.head(3)


{'P': 20, 'V': 30}


,ID,ride_time,ride_lat,ride_lon,alight_lat,alight_lon
0,15,0,37.428604,127.141289,37.369213,127.231789
1,7835,0,37.383121,127.120571,37.368794,127.141646
2,25,0,37.484272,127.104581,37.386928,127.074041


In [40]:
passengers

,ID,ride_time,ride_lat,ride_lon,alight_lat,alight_lon
0,15,0,37.428604,127.141289,37.369213,127.231789
1,7835,0,37.383121,127.120571,37.368794,127.141646
2,25,0,37.484272,127.104581,37.386928,127.074041
3,7856,0,37.499676,127.111877,37.375553,127.112617
4,34,0,37.396011,127.111038,37.393070,127.232033
5,36,0,37.397079,127.112129,37.260082,127.214142
6,5269,0,37.498304,127.028803,37.396205,127.114631
7,5243,0,37.442952,127.135766,37.435057,127.165954
8,14712,0,37.440375,127.149744,37.449192,127.158953
9,5234,0,37.433162,127.129110,37.459848,127.126534


In [38]:
# 4) 비용행렬 + 매칭
P, V = len(passengers), len(vehicles)
cost = np.zeros((P, V), dtype=float)
for i in range(P):
    plat, plon = passengers.loc[i, 'ride_lat'], passengers.loc[i, 'ride_lon']
    for j in range(V):
        vlat, vlon = vehicles.loc[j, 'lat'], vehicles.loc[j, 'lon']
        cost[i, j] = haversine_km(plat, plon, vlat, vlon)

# 유효성 보정
if not np.isfinite(cost).all():
    cost = np.where(np.isfinite(cost), cost, 1e6/2)

matches = solve_assignment_ortools(cost)
print('matches:', matches[:10])


{'backend': 'CBC_MIXED_INTEGER_PROGRAMMING', 'status': 0, 'P': 20, 'V': 30, 'N': 30, 'matched': 20}
matches: [(0, 5), (1, 1), (2, 8), (3, 22), (4, 15), (5, 10), (6, 0), (7, 13), (8, 21), (9, 7)]


In [39]:
# 5) trips.json / passengers.json 생성 및 저장
TRIPS = []
PASSENGER_MARKERS = []
SPEED_KM_PER_H = 30.0

for (pi, vj) in matches:
    p = passengers.loc[pi]
    v = vehicles.loc[vj]
    start_time = max(float(v['start_time']), float(p['ride_time']))

    route1, ts1 = build_simple_route_with_timestamp([
        (v['lon'], v['lat']),
        (p['ride_lon'], p['ride_lat'])
    ], speed_km_per_h=SPEED_KM_PER_H, start_time_min=start_time)

    route2, ts2 = build_simple_route_with_timestamp([
        (p['ride_lon'], p['ride_lat']),
        (p['alight_lon'], p['alight_lat'])
    ], speed_km_per_h=SPEED_KM_PER_H, start_time_min=ts1[-1])

    full_route = route1 + route2[1:]
    full_ts = ts1 + ts2[1:]

    TRIPS.append({'route': full_route, 'timestamp': full_ts})
    PASSENGER_MARKERS.append({'passenger_id': int(p['ID']), 'location': [float(p['ride_lon']), float(p['ride_lat'])], 'timestamp': [float(p['ride_time']), float(ts1[-1])]})

PUBLIC_DATA_DIR = os.path.join('simulation_vis', 'public', 'data')
os.makedirs(PUBLIC_DATA_DIR, exist_ok=True)

with open(os.path.join(PUBLIC_DATA_DIR, 'trips.json'), 'w', encoding='utf-8') as f:
    json.dump(TRIPS, f)
with open(os.path.join(PUBLIC_DATA_DIR, 'passengers.json'), 'w', encoding='utf-8') as f:
    json.dump(PASSENGER_MARKERS, f)

print({'trips': len(TRIPS), 'passengers': len(PASSENGER_MARKERS)})


{'trips': 20, 'passengers': 20}


# 6) React 뷰어 실행 안내
- PowerShell:
```bash
cd simulation_vis
npm install
npm start
```
- 브라우저 `http://localhost:3000` → 노란 경로(TripsLayer), 파란 점(승객). 픽업 완료 시 승객 점 사라짐.


## 7) 기말 프로젝트 연습

시각화 결과가 나오나요? 택시들이 하늘을 날아다니는 것을 볼 수 있습니다. 아래를 고민해보세요.

1. 어떻게 하면 택시가 길을 따라 이동하도록 만들 수 있을까요? (이전 실습시간에 이 문제를 다뤘습니다). 도전해 봅시다.
2. 이 시뮬레이션에서 택시의 서비스 수준을 어떻게 평가할 수 있을까요? 가장 대표적인 지표를 생각해보고, 직접 뽑아봅시다. 
3. 실제 택시의 경우 매분 새로운 승객이 택시를 호출할 것입니다. 이를 어떻게 하면 구현할 수 있을까요? `passengers` 데이터프레임 아니라 `passengers_raw` 데이터에서 매분마마택시를 배차해보고, 택시를 운영해보세요.